<a href="https://colab.research.google.com/github/minjeon99/5th-Basic-ML/blob/master/9_6_%EC%84%B8%EC%85%98_%EB%B6%84%EB%A5%98_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **분류 연습 문제**
___
출처 : 핸즈온 머신러닝 Ch03 분류 연습문제 1, 2번

In [1]:
# import data
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version = 1, as_frame = False)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
X, y = mnist["data"], mnist["target"]

In [3]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

### **1. MNIST 데이터셋으로 분류기를 만들어 테스트 세트에서 97% 정확도를 달성해보세요.**
___

1. `KNeghtborsClassifier`를 사용하는 것을 추천합니다.
2. `weights`와 `n_neighbors` 하이퍼 파라미터로 그리드 탐색을 시도하여, 좋은 하이퍼 파라미터 값을 찾아보세요.

In [10]:
# import package
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

In [17]:
knn = KNeighborsClassifier()

grid_params = {
    'n_neighbors' : [4,5,6],
    'weights' : ['distance']
}

grid = GridSearchCV(knn, grid_params, cv=3)
grid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [4, 5, 6], 'weights': ['distance']})

In [18]:
# best hyperparameter
print("Best Parameters : ", grid.best_params_)

Best Parameters :  {'n_neighbors': 4, 'weights': 'distance'}


In [19]:
# best score
print("Best Score : ", grid.best_score_)

Best Score :  0.9703500000000002


In [20]:
# model test
print("Best Test Score : ", grid.score(X_test, y_test))

Best Test Score :  0.9714


### **2. 다음 단계를 따라 인위적으로 훈련 세트를 늘리는 데이터 증식 또는 훈련 세트 확장 기법을 연습해봅시다.**
___

#### **STEP 1. MNIST 이미지를 (왼, 오른, 위, 아래) 어느 방향으로든 한 픽셀 이동시킬 수 있는 함수를 만들어 보세요.**

In [30]:
#이미지를 동서남북 방향으로 shift 할 scipy 모듈 import
from scipy.ndimage.interpolation import shift
import numpy as np

<ipython-input-30-cd5e9928518e>:2: DeprecationWarning: Please import `shift` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.interpolation import shift


In [28]:
#이미지를 reshape 해서 이미지를  dx, dy 만큼 변환한 뒤 원래 형태로 반환하는 함수
def shift_image(image, dx, dy):
    image = image.reshape((28,28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant") #cval : 이동 후에 새롭게 생성되는 픽셀의 값을 채우는 데 사용되는 상수값
    return shifted_image.reshape([-1])

####  **STEP 2. 앞에서 만든 함수를 이용하여, 훈련 세트에 있는 각 이미지에 대해 네 개의 이동된 복사본(방향마다 한 개씩)을 만들어 훈련 세트에 추가하세요**

In [31]:
X_train_augmented=[image for image in X_train]
y_train_augmented=[label for label in y_train]

for dx, dy in ((1,0), (-1,0), (0,1), (0,-1)):
    for image, label in zip(X_train, y_train):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)

X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

####  **STEP 3. 위에서 확장한 데이터셋을 이용하여, 1번 문제에서 찾은 최적 모델을 훈련시키고, 테스트 세트에서 정확도를 측정해보세요**

In [34]:
from sklearn.model_selection import GridSearchCV

knn_clf=KNeighborsClassifier(grid.best_params_)

In [35]:
knn_clf.fit(X_train_augmented,y_train_augmented)

InvalidParameterError: The 'n_neighbors' parameter of KNeighborsClassifier must be an int in the range [1, inf) or None. Got {'n_neighbors': 4, 'weights': 'distance'} instead.

In [36]:
from sklearn.metrics import accuracy_score

y_pred=knn_clf.predict(X_test)
accuracy_score(y_test, y_pred)

NotFittedError: This KNeighborsClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.